In [1]:
import torch.onnx
import torchvision.models as models
import onnx

In [2]:
# Load saved pytorch model
model_path = 'model_mobilenet_v2.pth'
model = models.mobilenet_v2()
model.load_state_dict(torch.load(model_path))
model.eval()

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [3]:
# Export the trained model to ONNX
batch_size = 1
x = torch.randn(batch_size, 3, 256, 256, requires_grad=True)
torch.onnx.export(model, x, "model_mobilenet_v2.onnx", export_params=True, opset_version=10, do_constant_folding=True, input_names = ['input'], output_names = ['output'], dynamic_axes={'input' : {0 : 'batch_size'}, 'output' : {0 : 'batch_size'}})

RuntimeError: Given groups=1, weight of size [32, 3, 3, 3], expected input[1, 1, 256, 256] to have 3 channels, but got 1 channels instead

In [6]:
# Check the ONNX model
onnx_model = onnx.load("model_mobilenet_v2.onnx")
onnx.checker.check_model(onnx_model)

In [19]:
import time
import cv2
import numpy as np
from openvino.preprocess import PrePostProcessor, ResizeAlgorithm
from openvino.runtime import Core, Layout, Type

In [12]:
model_path = 'model_mobilenet_v2.onnx'
image_path = 'test.png'
device_name = 'MYRIAD'

In [13]:
# Initialize openvino runtime core
print('Creating OpenVINO Runtime Core')
core = Core()

In [14]:
# Read ONNX model
print(f'Reading the model: {model_path}')
# (.xml and .bin files) or (.onnx file)
model = core.read_model(model_path)

In [15]:
# Read input image
image = cv2.imread(image_path)
# Add N dimension
input_tensor = np.expand_dims(image, 0)

In [16]:
# Preprocess input image
ppp = PrePostProcessor(model)

_, h, w, _ = input_tensor.shape

# 1) Set input tensor information:
# - input() provides information about a single model input
# - reuse precision and shape from already available `input_tensor`
# - layout of data is 'NHWC'
ppp.input().tensor() \
    .set_shape(input_tensor.shape) \
    .set_element_type(Type.u8) \
    .set_layout(Layout('NHWC'))  # noqa: ECE001, N400

# 2) Adding explicit preprocessing steps:
# - apply linear resize from tensor spatial dims to model spatial dims
ppp.input().preprocess().resize(ResizeAlgorithm.RESIZE_LINEAR)

# 3) Here we suppose model has 'NCHW' layout for input
ppp.input().model().set_layout(Layout('NCHW'))

# 4) Set output tensor information:
# - precision of tensor is supposed to be 'f32'
ppp.output().tensor().set_element_type(Type.f32)

# 5) Apply preprocessing modifying the original 'model'
model = ppp.build()

In [21]:
# Load the model into the device
print(f'Loading the model to the {device_name} device')
compiled_model = core.compile_model(model, device_name)

In [32]:
# Create infer request
print('Starting inference in synchronous mode')
iterations = 100
start = time.time()
for i in range(iterations):
    results = compiled_model.infer_new_request({0: input_tensor})
end = time.time()
average_time = (end - start) / iterations
print(f'Average processing time: {average_time}s')

Starting inference in synchronous mode
Inference completed successfully. Processing time: 0.08526024341583252s
Processing_time:  0.08526024341583252


In [33]:
# Get prediction
predictions = next(iter(results.values()))
probs = predictions.reshape(-1)
top = np.argmax(probs)
labels = ["Cloud", "Edge", "Good"]
if top < len(labels):
    print(f"Prediction: {labels[top]}")
else:
    print(f"Prediction Unknown, Output: {top}")

Prediction: Cloud
